In [1]:
from __future__ import print_function
from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Activation
from keras.activations import softmax as Softmax
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from nltk.translate.bleu_score import corpus_bleu
from keras.utils import to_categorical
from keras import backend as K
import numpy as np
import unicodedata
import re

Using TensorFlow backend.


In [0]:
BATCH_SIZE = 128  # Batch size for training
NUM_SAMPLES = 12500
EPOCHS = 50
OPTIMIZER = "adam"
EMBED_DIM = 100
HIDDEN_DIM = 256
DATA_PATH = 'deu.txt'

In [0]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')

In [0]:
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile, 'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print("Done.", len(model), " words loaded!")
    return model

In [0]:
def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())

    # creating a space between words and punctation following it
    # eg: "he is a boy." => "he is a boy ."
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

    w = w.rstrip().strip()

    # adding a start and an end token to the sequence
    # so that the model know when to start and stop predicting
    w = "<SOS> " + w + " <EOS>"
    return w

In [6]:
eng_embedding = loadGloveModel('glove.6B.100d.txt')

Loading Glove Model
Done. 400000  words loaded!


In [7]:
ger_embedding = loadGloveModel('embed_tweets_de_100D_fasttext')

Loading Glove Model
Done. 533797  words loaded!


In [0]:
input_texts = []
target_texts = []
target_words = set()

with open(DATA_PATH, 'r', encoding='utf-8') as f:
    lines = f.read().split("\n")
    for line in lines[:NUM_SAMPLES]:
        input_text, target_text = line.split('\t')
        input_text = preprocess_sentence(input_text)
        target_text = preprocess_sentence(target_text)
        input_texts.append(input_text)
        target_texts.append(target_text)
        target_words.update(target_text.split())

target_words = sorted(list(target_words))

In [0]:
max_input_seqlen = max([len(txt.split()) for txt in input_texts])
max_target_seqlen = max([len(txt.split()) for txt in target_texts])
target_vocab_size = len(target_words) + 1

In [0]:
# get decoder input data
decoder_data = []
for text in target_texts:
    tmp = []
    for word in text.split():
        embed = np.random.randn(EMBED_DIM)
        if word in ger_embedding:
            embed = ger_embedding[word]
        tmp.append(embed)
    decoder_data.append(tmp)
decoder_data = pad_sequences(decoder_data, max_target_seqlen, padding="post")

In [0]:
# get decoder target data
targword2idx = dict([(word, i + 1) for i, word in enumerate(target_words)])
idx2targword = dict((i, word) for word, i in targword2idx.items())
decoder_target_data = []
for text in target_texts:
    tmp = []
    for idx, word in enumerate(text.split()):
        if idx > 0:
            tmp.append(targword2idx[word])
    decoder_target_data.append(tmp)
decoder_target_data = pad_sequences(
    decoder_target_data, max_target_seqlen, padding="post")
decoder_target_data = to_categorical(decoder_target_data)

In [0]:
# get encoder data
encoder_data = []
for text in input_texts:
    tmp = []
    for word in text.split():
        embed = np.random.randn(EMBED_DIM)
        if word in eng_embedding:
            embed = eng_embedding[word]
        tmp.append(embed)
    encoder_data.append(tmp)
encoder_data = pad_sequences(encoder_data, max_input_seqlen, padding="post")

In [13]:
encoder_data.shape

(12500, 9, 100)

In [14]:
decoder_data.shape

(12500, 14, 100)

In [15]:
decoder_target_data.shape

(12500, 14, 4223)

In [0]:
# construct model
encoder_inputs = Input(shape=(max_input_seqlen, EMBED_DIM))
encoder_lstm = LSTM(HIDDEN_DIM, return_state=True, name="encoder_lstm")
_, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, EMBED_DIM))
decoder_lstm = LSTM(HIDDEN_DIM, return_sequences=True,
                    return_state=True, name="decoder_lstm")
decoder_outputs, _, _ = decoder_lstm(
    decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(
    target_vocab_size, activation="softmax", name="decoder_dense")
decoder_outputs = decoder_dense(decoder_outputs)

In [17]:
# define training model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer=OPTIMIZER,
              loss='categorical_crossentropy', metrics=["acc"])
print(model.summary())
filename = 'seq2seq_keras.h5'
checkpoint = ModelCheckpoint(
    filename, verbose=1, save_best_only=True, mode='min')
# checkpoint = ModelCheckpoint(filename, verbose=1, mode='min')
model.fit([encoder_data, decoder_data], decoder_target_data,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS, callbacks=[checkpoint], validation_split=0.2)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 9, 100)       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 100)    0                                            
__________________________________________________________________________________________________
encoder_lstm (LSTM)             [(None, 256), (None, 365568      input_1[0][0]                    
__________________________________________________________________________________________________
decoder_lstm (LSTM)             [(None, None, 256),  365568      input_2[0][0]                    
                                                                 encoder_lstm[0][1]               
          

/usr/local/lib/python3.6/dist-packages/keras/engine/network.py:877: UserWarning: Layer decoder_lstm was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'encoder_lstm/while/Exit_3:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'encoder_lstm/while/Exit_4:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


Epoch 2/50
10000/10000 [==============================] - 111s 11ms/step - loss: 1.4897 - acc: 0.7649 - val_loss: 1.6591 - val_acc: 0.7491

Epoch 00002: val_loss improved from 1.78619 to 1.65910, saving model to seq2seq_keras.h5
Epoch 3/50
10000/10000 [==============================] - 112s 11ms/step - loss: 1.3698 - acc: 0.7776 - val_loss: 1.5731 - val_acc: 0.7597

Epoch 00003: val_loss improved from 1.65910 to 1.57311, saving model to seq2seq_keras.h5
Epoch 4/50
10000/10000 [==============================] - 112s 11ms/step - loss: 1.2698 - acc: 0.7865 - val_loss: 1.4818 - val_acc: 0.7723

Epoch 00004: val_loss improved from 1.57311 to 1.48178, saving model to seq2seq_keras.h5
Epoch 5/50
10000/10000 [==============================] - 113s 11ms/step - loss: 1.1821 - acc: 0.7991 - val_loss: 1.4238 - val_acc: 0.7843

Epoch 00005: val_loss improved from 1.48178 to 1.42381, saving model to seq2seq_keras.h5
Epoch 6/50
10000/10000 [==============================] - 111s 11ms/step - loss: 1.1

In [0]:
# create inference model
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(HIDDEN_DIM,))
decoder_state_input_c = Input(shape=(HIDDEN_DIM,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
# decoder_outputs = (BATCH_SIZE, seqlen, HIDDEN_DIM)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
# decoder_outputs = (BATCH_SIZE, seqlen, target_token_size)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [0]:
def decode(input_seq):
    states = encoder_model.predict(input_seq)
    target_seq = np.random.randn(EMBED_DIM)
    target_seq = [[target_seq]]
    stop_condition = False
    prediction = ''

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states)
        sampled_token_idx = np.argmax(output_tokens[0, -1, :])
        sampled_word = idx2targword[sampled_token_idx]
        prediction += sampled_word + " "

        if (sampled_word == '<EOS>' or len(prediction) > max_target_seqlen):
            stop_condition = True

        if sampled_word in ger_embedding:
            target_seq = ger_embedding[sampled_word]
        else:
            target_seq = np.random.randn(EMBED_DIM)
        target_seq = [[target_seq]]
        states = [h, c]

    return prediction

In [20]:
actual, predicted = list(), list()

for index in range(1000):
    input_seq = encoder_data[index]
    input_seq = np.expand_dims(input_seq, axis=0)
    actual.append(target_texts[index].split())
    prediction = decode(input_seq)
    predicted.append(prediction.split())
    print('-')
    print("Input sentence: ", input_texts[index])
    print("Translation: ", prediction)
    print("Truth value: ", target_texts[index])

print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
print('BLEU-2: %f' %
      corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
print('BLEU-3: %f' %
      corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
print('BLEU-4: %f' %
      corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

-
Input sentence:  <SOS> hi . <EOS>
Translation:  hallo ! <EOS> 
Truth value:  <SOS> hallo ! <EOS>
-
Input sentence:  <SOS> hi . <EOS>
Translation:  fangen sie an bord 
Truth value:  <SOS> gru gott ! <EOS>
-
Input sentence:  <SOS> run ! <EOS>
Translation:  lauf ! <EOS> 
Truth value:  <SOS> lauf ! <EOS>
-
Input sentence:  <SOS> fire ! <EOS>
Translation:  feuer ! <EOS> 
Truth value:  <SOS> feuer ! <EOS>
-
Input sentence:  <SOS> help ! <EOS>
Translation:  hor auf ! <EOS> 
Truth value:  <SOS> hilfe ! <EOS>
-
Input sentence:  <SOS> help ! <EOS>
Translation:  zu hulf ! <EOS> 
Truth value:  <SOS> zu hulf ! <EOS>
-
Input sentence:  <SOS> stop ! <EOS>
Translation:  hau ab ! <EOS> 
Truth value:  <SOS> stopp ! <EOS>
-
Input sentence:  <SOS> wait ! <EOS>
Translation:  warte ! <EOS> 
Truth value:  <SOS> warte ! <EOS>
-
Input sentence:  <SOS> go on . <EOS>
Translation:  mach weiter . <EOS> 
Truth value:  <SOS> mach weiter . <EOS>
-
Input sentence:  <SOS> hello ! <EOS>
Translation:  hallo ! <EOS> 
Tr

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


BLEU-1: 0.140380
BLEU-2: 0.374674
BLEU-3: 0.554871
BLEU-4: 0.612106


In [0]:
def predict_sent(input_text):
    input_seq = []
    tmp = []
    text = preprocess_sentence(input_text)
    for word in text.split():
        embed = np.random.randn(EMBED_DIM)
        if word in eng_embedding:
            embed = eng_embedding[word]
        tmp.append(embed)
    input_seq.append(tmp)
    input_seq = pad_sequences(input_seq, max_input_seqlen, padding="post")
    states = encoder_model.predict(input_seq)
    target_seq = np.random.randn(EMBED_DIM)
    target_seq = [[target_seq]]
    target_text = ''
    stop_condition = False
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states)

        sampled_token_idx = np.argmax(output_tokens[0, -1, :])
        sampled_word = idx2targword[sampled_token_idx]
        target_text += sampled_word + " "

        if sampled_word == '\n' or len(target_text) >= max_target_seqlen:
            stop_condition = True

        if sampled_word in ger_embedding:
            target_seq = ger_embedding[sampled_word]
        else:
            target_seq = np.random.randn(EMBED_DIM)
        target_seq = [[target_seq]]

        states = [h, c]
    return target_text.strip()

In [45]:
predict_sent("She is beautiful")

'sie ist schon'